In [41]:
import pandas as pd
import datetime

# Import data

In [90]:
df_client=pd.read_csv("./raw_data/client.csv")
df_electricity=pd.read_csv("./raw_data/electricity_prices.csv")
df_gas=pd.read_csv("./raw_data/gas_prices.csv")



# Changing data format

### Manipulating dates
Check the number of unique values for the date column in the electricity
By checking only the unique dates, it will reduce the number of calculation when we will compare the date between the electricity df and the train df

In [91]:
print("Number of unique values in elec :", df_electricity["origin_date"].nunique())
print("Number of unique values in gas :", df_gas["origin_date"].nunique())

Number of unique values in elec : 15286
Number of unique values in gas : 637


In [44]:
# # Creating a new column "date" with the French formated date 
# # For elec
# df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
# df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%y-%m-%d"))
# print(df_electricity["date"])
# 
# # For gases
# df_gas["date"] = pd.to_datetime(df_gas["forecast_date"])
# df_gas["date"] = df_gas["date"].apply(lambda x : x.strftime("%y-%m-%d"))
# print(df_gas["date"])

# Feature Engineering

### Adding prices in df_train
First of all we can see that the dataframe for the gas, do not use the same date format than the electricity and the train dataframes date format.
So we will be able to merge the info from the electricity dataframe, based on the datetime from the train dataframe.
But for the gas dataframe, we will need to add a formated column in train df. A column with the same format as the date in the gas df. We keep the date, we just remove the time.
After this manipulation, we can merge the gas prices too.

In [97]:
# Import train data
df_train = pd.read_csv("./raw_data/train.csv").sample(10000)

## Merge elec prices
# Creating a new column based on datetime
df_train["origin_date"] = df_train["datetime"]
# Selecting the infos we need to merge in the electricity df
df_elec_prices = df_electricity.filter(['euros_per_mwh', 'origin_date'], axis=1)
# Merging
df_train = df_train.merge(df_elec_prices, how="left", on="origin_date")

## Merge gas prices
# Removing the hours in the origin_date column created above
df_train["origin_date"] = df_train["origin_date"].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%d"))
# Selecting the infos we need to merge in the gas df
df_gas_prices = df_gas.filter(['lowest_price_per_mwh', 'highest_price_per_mwh', 'origin_date'])
# Merging
df_train = df_train.merge(df_gas_prices, how="left", on="origin_date")

# df_train.describe()
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,origin_date,euros_per_mwh,lowest_price_per_mwh,highest_price_per_mwh
0,14,0,3,62.130,1,2021-10-10 16:00:00,39,117185,54,2021-10-10,145.07,67.00,67.83
1,8,1,3,327.095,1,2023-04-21 18:00:00,597,1890013,33,2023-04-21,44.74,44.00,48.40
2,10,1,2,0.000,0,2023-01-17 16:00:00,503,1591274,41,2023-01-17,150.11,51.00,71.07
3,0,1,2,99.963,1,2023-02-03 22:00:00,520,1646239,61,2023-02-03,131.62,61.50,65.10
4,12,1,3,638.699,1,2022-04-14 17:00:00,225,695253,49,2022-04-14,68.39,94.78,111.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0,3,17.792,1,2022-08-19 05:00:00,352,1100405,7,2022-08-19,177.06,225.00,240.00
9996,11,1,3,3691.097,1,2022-10-28 01:00:00,422,1325763,48,2022-10-28,92.77,88.29,122.00
9997,4,1,3,0.000,0,2023-04-04 22:00:00,580,1837316,18,2023-04-04,67.76,50.00,55.47
9998,9,0,3,182.641,0,2022-05-14 14:00:00,255,792080,35,2022-05-14,7.04,100.00,110.00
